# Day 11 - Monkey in the Middle

## Data

In [1]:
example_data = """
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
""".strip()

print(example_data)

Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1


In [2]:
import aocd
raw_data = aocd.get_data(year=2022, day=11)
print(raw_data)

Monkey 0:
  Starting items: 89, 73, 66, 57, 64, 80
  Operation: new = old * 3
  Test: divisible by 13
    If true: throw to monkey 6
    If false: throw to monkey 2

Monkey 1:
  Starting items: 83, 78, 81, 55, 81, 59, 69
  Operation: new = old + 1
  Test: divisible by 3
    If true: throw to monkey 7
    If false: throw to monkey 4

Monkey 2:
  Starting items: 76, 91, 58, 85
  Operation: new = old * 13
  Test: divisible by 7
    If true: throw to monkey 1
    If false: throw to monkey 4

Monkey 3:
  Starting items: 71, 72, 74, 76, 68
  Operation: new = old * old
  Test: divisible by 2
    If true: throw to monkey 6
    If false: throw to monkey 0

Monkey 4:
  Starting items: 98, 85, 84
  Operation: new = old + 7
  Test: divisible by 19
    If true: throw to monkey 5
    If false: throw to monkey 7

Monkey 5:
  Starting items: 78
  Operation: new = old + 8
  Test: divisible by 5
    If true: throw to monkey 3
    If false: throw to monkey 0

Monkey 6:
  Starting items: 86, 70, 60, 88, 8

In [3]:
from dataclasses import dataclass
from typing import Callable

@dataclass
class MonkeyBusinessRules:
    debug: bool
    divide_worry_level: bool
    modulus: int

        
@dataclass
class ItemToss(object):
    """Represents one monkey throwing an item to another monkey."""
    item: int
    target: int
            

@dataclass
class Monkey(object):
    number: int
    items: list[int]
    operation: Callable[[int], int]
    throw_to: Callable[[int], int]
    items_inspected: int = 0    
        
    def __repr__(self):
        return f" Monkey {self.number} Has items {self.items}"

    def catch_item(self, item):
        self.items.append(item)
    
    def inspect_item(self, rules):
        """Changes the worry level of the first item and determines which monkey to toss it to."""
        self.items_inspected += 1
        
        inspectee = self.items.pop(0)
        
        if rules.debug:
            print(f"Monkey {self.number} is inspecting item with worry level {inspectee}")

        inspectee = self.operation(inspectee)
        if rules.divide_worry_level:
            inspectee //= 3
        else:
            inspectee %= rules.modulus
            
        target = self.throw_to(inspectee)
        
        if rules.debug:
            print(f"  Monkey {self.number} throws item {inspectee} to monkey {target}!")
        
        return ItemToss(inspectee, target)
    
    def play_turn(self, rules):
        while len(self.items) > 0:
            yield self.inspect_item(rules)

## Parsing

In [4]:
import pyparsing as pp

integer = pp.Word(pp.nums)
        
@integer.set_parse_action
def parse_integer(tokens):
    return int(tokens[0])

identifier = pp.Word(pp.alphas)
term = identifier | integer
@term.set_parse_action
def parse_term(tokens):
    """Tokens is either an integer or a name of a  variable to lookup"""
    def resolve_identifier(identifiers):
        if type(tokens[0]) == int:
            return tokens[0]
        else:
            return identifiers[tokens[0]]
    return resolve_identifier


operators = {
    '+': lambda left, right: left + right,
    '-': lambda left, right: left - right,
    '*': lambda left, right: left * right,
    '/': lambda left, right: left / right
}

operator = pp.one_of("+ - * /")
@operator.set_parse_action
def parse_operator(tokens):
    op = tokens[0]
    return operators[op]


binary_expression = term + operator + term
@binary_expression.set_parse_action
def binary_expression(tokens):
    def calculate(identifiers):
        left = tokens[0](identifiers)
        op = tokens[1]
        right = tokens[2](identifiers)
        return op(left, right)

    return calculate

test_binary_expression = binary_expression.parse_string("foo * bar")[0]
assert test_binary_expression({
    "foo": 3,
    "bar": 5
}) == 15

In [5]:
monkey_number = pp.Suppress("Monkey") + integer + pp.Suppress(":")
monkey_number.run_tests("""
Monkey 3:
Monkey 5:
Monkey18:
""")

integer_list = pp.delimited_list(integer)
starting_items = pp.Suppress("Starting items:") + integer_list

@starting_items.set_parse_action
def parse_starting_items(tokens):
    return tuple(tokens)

starting_items.run_tests("""
Starting items: 1, 2, 3
Starting items: 4,5
""")


operation = pp.Suppress("Operation: new = ") + binary_expression

@operation.set_parse_action
def parse_operation(tokens):
    """
    Return a function that given the old worry level, returns the new worry level.
    """
    def operation(old):
        return tokens[0]({"old": old})
    return operation

test_operation = operation.parse_string("Operation: new = old + old")[0]

assert test_operation(5) == 10

condition = pp.Suppress("Test: divisible by") + integer
true_case = pp.Suppress("If true: throw to monkey") + integer
false_case = pp.Suppress("If false: throw to monkey") + integer
throw_to = condition + true_case + false_case

@throw_to.set_parse_action
def parse_throw_to(tokens):
    """
    Returns a function that takes the worry level and returns the monkey to throw the item to.
    tokens[0] = condition to check worry level against
    tokens[1] = true case
    tokens[2] = false case
    """
    def throw_to(worry_level):
        if worry_level % tokens[0] == 0:
            return tokens[1]
        return tokens[2]
    
    throw_to.modulus = tokens[0]
    return throw_to

test_throw_to = throw_to.parse_string("""
Test: divisible by 11
    If true: throw to monkey 1
    If false: throw to monkey 2
""")[0]

assert test_throw_to(22) == 1
assert test_throw_to(1) == 2

monkey = monkey_number + starting_items + operation + throw_to

@monkey.set_parse_action
def parse_monkey(tokens):
    return Monkey(
        number = tokens[0],
        items = list(tokens[1]),
        operation = tokens[2],
        throw_to = tokens[3]
    )

monkey_language = pp.ZeroOrMore(monkey)


def parse(data):
    return monkey_language.parse_string(data).as_list()

example_monkeys = parse(example_data)

real_monkeys = parse(raw_data)

example_monkeys


Monkey 3:
[3]

Monkey 5:
[5]

Monkey18:
[18]

Starting items: 1, 2, 3
[(1, 2, 3)]

Starting items: 4,5
[(4, 5)]


[ Monkey 0 Has items [79, 98],
  Monkey 1 Has items [54, 65, 75, 74],
  Monkey 2 Has items [79, 60, 97],
  Monkey 3 Has items [74]]

## Part 1

In [6]:
def play_round(monkeys, rules):
    for monkey in monkeys:
        for toss in monkey.play_turn(rules):
            catcher = monkeys[toss.target]
            catcher.catch_item(toss.item)


def calculate_monkey_business(monkeys, num_rounds = 20, debug = False, divide_worry_level = True):
    """
    Run num_rounds of Monkey in the Middle, return the product of the number of items inspected by
    the 2 most active monkeys.
    """
    
    # Calculate the modulus to adjust the worry level by calculating a multiple of all
    # monkey's modulus value.
    modulus = 1
    for monkey in monkeys:
        modulus *= monkey.throw_to.modulus
    
    print(f"Adjust monkey business by {modulus}")
    
    rules = MonkeyBusinessRules(debug, divide_worry_level, modulus = modulus)
    
    
    for i in range(num_rounds):
        play_round(monkeys, rules)
        if i % 1000 == 0:
            print(f"Ran {i} rounds")
        
    monkeys.sort(key = lambda monkey: monkey.items_inspected, reverse = True)
    
    print(f"The most active monkey is {monkeys[0]}, and then monkey{monkeys[1]}")
    
    return monkeys[0].items_inspected * monkeys[1].items_inspected

assert calculate_monkey_business(example_monkeys, debug = True) == 10605

Adjust monkey business by 96577
Monkey 0 is inspecting item with worry level 79
  Monkey 0 throws item 500 to monkey 3!
Monkey 0 is inspecting item with worry level 98
  Monkey 0 throws item 620 to monkey 3!
Monkey 1 is inspecting item with worry level 54
  Monkey 1 throws item 20 to monkey 0!
Monkey 1 is inspecting item with worry level 65
  Monkey 1 throws item 23 to monkey 0!
Monkey 1 is inspecting item with worry level 75
  Monkey 1 throws item 27 to monkey 0!
Monkey 1 is inspecting item with worry level 74
  Monkey 1 throws item 26 to monkey 0!
Monkey 2 is inspecting item with worry level 79
  Monkey 2 throws item 2080 to monkey 1!
Monkey 2 is inspecting item with worry level 60
  Monkey 2 throws item 1200 to monkey 3!
Monkey 2 is inspecting item with worry level 97
  Monkey 2 throws item 3136 to monkey 3!
Monkey 3 is inspecting item with worry level 74
  Monkey 3 throws item 25 to monkey 1!
Monkey 3 is inspecting item with worry level 500
  Monkey 3 throws item 167 to monkey 1!
M

In [11]:
calculate_monkey_business(real_monkeys)

Adjust monkey business by 9699690
Ran 0 rounds
The most active monkey is  Monkey 7 Has items [], and then monkey Monkey 4 Has items []


119715

## Part 2

In [12]:
example_monkeys = parse(example_data)
assert calculate_monkey_business(example_monkeys, num_rounds=10000, divide_worry_level=False) == 2713310158

real_monkeys = parse(raw_data)
calculate_monkey_business(real_monkeys, num_rounds = 10000, divide_worry_level = False)

Adjust monkey business by 96577
Ran 0 rounds
Ran 1000 rounds
Ran 2000 rounds
Ran 3000 rounds
Ran 4000 rounds
Ran 5000 rounds
Ran 6000 rounds
Ran 7000 rounds
Ran 8000 rounds
Ran 9000 rounds
The most active monkey is  Monkey 0 Has items [63602, 56040, 11941, 10573, 61607], and then monkey Monkey 3 Has items []
Adjust monkey business by 9699690
Ran 0 rounds
Ran 1000 rounds
Ran 2000 rounds
Ran 3000 rounds
Ran 4000 rounds
Ran 5000 rounds
Ran 6000 rounds
Ran 7000 rounds
Ran 8000 rounds
Ran 9000 rounds
The most active monkey is  Monkey 4 Has items [], and then monkey Monkey 2 Has items [6438397, 919244, 8576894, 8066384, 2523704, 8066384, 919244, 919244, 6170204, 2543000, 355100, 2543000, 9325490, 6627080, 6627080, 1157330]


18085004878